In [1]:
import pandas as pd
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
conn_url = os.getenv("DB_CONN")

In [12]:
users = pd.read_sql(
    "SELECT * FROM public.user_data;",
    conn_url
)

posts = pd.read_sql(
    "SELECT * FROM public.post_text_df;",
    conn_url
)

In [3]:
feedback = pd.read_sql(
    """WITH RankedFeedData AS (
        SELECT 
            timestamp,
            user_id,
            post_id,
            action,
            target,
            ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY timestamp DESC) AS row_num
        FROM 
            feed_data
    )
    SELECT 
        timestamp,
        user_id,
        post_id,
        action,
        target
    FROM 
        RankedFeedData
    WHERE 
        row_num <= 30;
    """,
    conn_url
)

In [4]:
users_likes = pd.read_sql(
    """WITH RankedFeedData AS (
        SELECT 
            timestamp,
            user_id,
            post_id,
            action,
            target,
            ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY timestamp DESC) AS row_num
        FROM 
            feed_data
        WHERE
            action = 'like'
    )
    SELECT 
        timestamp,
        user_id,
        post_id,
        action,
        target
    FROM 
        RankedFeedData
    WHERE 
        row_num <= 10;
    """,
    conn_url
)

In [5]:
users_views = pd.read_sql(
    """WITH RankedFeedData AS (
        SELECT 
            timestamp,
            user_id,
            post_id,
            action,
            target,
            ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY timestamp DESC) AS row_num
        FROM 
            feed_data
        WHERE
            action = 'view'
    )
    SELECT 
        timestamp,
        user_id,
        post_id,
        action,
        target
    FROM 
        RankedFeedData
    WHERE 
        row_num <= 10;
    """,
    conn_url
)

In [6]:
users_likes.shape, users_views.shape

((1615528, 5), (1632050, 5))

In [7]:
# удалим просмотры пользователя у тех постов, которых пользователь поставил лайк

df_like_copy = users_likes.copy()
df_view_copy = users_views.copy()

merged_df = pd.merge(df_view_copy, df_like_copy, on=['user_id', 'post_id'], how='left', suffixes=('_view', '_like'))

filtered_df = merged_df[merged_df['action_like'].isnull()]

df_view_copy = filtered_df[['timestamp_view', 'user_id', 'post_id', 'action_view', 'target_view']]

df_view_copy = df_view_copy.rename(columns={'timestamp_view': 'timestamp', 'action_view': 'action', 'target_view': 'target'})


In [8]:
df_like_copy.shape, df_view_copy.shape

((1615528, 5), (1510221, 5))

In [9]:
feedback = pd.concat((df_view_copy, df_like_copy), axis=0)

In [10]:
feedback = feedback.drop(["target"], axis=1)
feedback["action"] = feedback["action"].apply(lambda x: 1 if x == "like" else 0)
feedback.head()

,timestamp,user_id,post_id,action
0,2021-12-29 15:24:59,200,1773,0
2,2021-12-29 15:23:54,200,1122,0
3,2021-12-29 15:23:29,200,1362,0
4,2021-12-29 15:21:53,200,1541,0
5,2021-12-29 15:21:18,200,2160,0


# не нужное

In [38]:
feedback = pd.read_csv("feed_back_each_user.csv")

In [5]:
feedback.head()

,timestamp,user_id,post_id,action,target
0,2021-12-29 15:24:59,200,1773,view,0
1,2021-12-29 15:24:57,200,2213,like,0
2,2021-12-29 15:24:31,200,2213,view,1
3,2021-12-29 15:23:54,200,1122,view,0
4,2021-12-29 15:23:29,200,1362,view,0


In [47]:
feedback.shape

(4896150, 5)

In [63]:
len(feedback[feedback["action"] == "like"]["user_id"].unique())

154073

In [59]:
# Удалим такие записи view, у которых есть дубликат, но с типом like. Так как из-за них модели сходят с ума, у нас есть две идентичные записи, но с разными таргетами
# + при делений на обучающую и тестовую выборки нужно делать шафл, так как у нас датасет отсортирован по юзерам, модель не видит тестовых юзеров вообще.
new_feed = feedback[:50000].copy()
for row in new_feed[new_feed["action"] == "like"].itertuples():
    user_part = new_feed[(new_feed["user_id"] == row.user_id) & (new_feed["post_id"] == row.post_id)]
    for index, n_row in user_part.iterrows():
        if n_row.action == "view":
            new_feed.drop(index, inplace=True)


In [60]:
new_feed.shape

(44612, 5)

# for ML

In [13]:
feedback = feedback.merge(posts).merge(users).sort_values("user_id")

In [14]:
feedback.head(1)

,timestamp,user_id,post_id,action,text,topic,gender,age,country,city,exp_group,os,source
0,2021-12-29 15:24:59,200,1773,0,Hearts 2-1 Livingston\n\nHearts wrapped up the...,sport,1,34,Russia,Degtyarsk,3,Android,ads


In [15]:
feedback.to_csv("data/full_data.csv")

KeyboardInterrupt: 

In [13]:
feedback.drop(["timestamp"], axis=1, inplace=True)

In [14]:
feedback.head(1)

,user_id,post_id,action,text,topic,gender,age,country,city,exp_group,os,source
0,200,1773,0,Hearts 2-1 Livingston\n\nHearts wrapped up the...,sport,1,34,Russia,Degtyarsk,3,Android,ads


In [15]:
feedback["k_words"] = feedback["text"].apply(lambda x: len(x.replace("\n", " ").split(" ")))

In [16]:
feedback.drop(["text"], axis=1, inplace=True)

In [17]:
feedback.head(1)

,user_id,post_id,action,topic,gender,age,country,city,exp_group,os,source,k_words
0,200,1773,0,sport,1,34,Russia,Degtyarsk,3,Android,ads,449


In [18]:
encoded_feedback = feedback.copy()
for col in ["topic", "country", "exp_group", "os", "source"]:
    one_hot = pd.get_dummies(encoded_feedback[col], prefix=col, drop_first=True).astype(int)
    encoded_feedback = pd.concat((encoded_feedback.drop(col, axis=1), one_hot), axis=1)

In [19]:
encoded_feedback.drop(["city"], axis=1, inplace=True)
encoded_feedback.head()

,user_id,post_id,action,gender,age,k_words,topic_covid,topic_entertainment,topic_movie,topic_politics,...,country_Russia,country_Switzerland,country_Turkey,country_Ukraine,exp_group_1,exp_group_2,exp_group_3,exp_group_4,os_iOS,source_organic
0,200,1773,0,1,34,449,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
17,200,4723,1,1,34,352,0,0,1,0,...,1,0,0,0,0,0,1,0,0,0
16,200,994,1,1,34,445,0,0,0,1,...,1,0,0,0,0,0,1,0,0,0
15,200,3567,1,1,34,26,1,0,0,0,...,1,0,0,0,0,0,1,0,0,0
14,200,7143,1,1,34,111,0,0,1,0,...,1,0,0,0,0,0,1,0,0,0


In [26]:
feedback.to_csv("new_feed_back.csv", sep=",", index=False)

In [5]:
# X = encoded_feedback.drop(["action"], axis=1)
# y = encoded_feedback["action"]
feedback = pd.read_csv("data/new_feed_back.csv", sep=",")
X = feedback.drop(["action"], axis=1)
y = feedback["action"]

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.15, random_state=32)

# Catboost

In [67]:
res_posts = posts.copy()

In [68]:
req_user = users[users["user_id"] == 200]

req_user["key"] = 0
res_posts["key"] = 0

pd.merge(res_posts, req_user, on='key').drop('key', axis=1)

C:\Users\ivana\AppData\Local\Temp\ipykernel_12888\2720941670.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  req_user["key"] = 0


,post_id,text,topic,user_id,gender,age,country,city,exp_group,os,source
0,1,UK economy facing major risks\n\nThe UK manufa...,business,200,1,34,Russia,Degtyarsk,3,Android,ads
1,2,Aids and climate top Davos agenda\n\nClimate c...,business,200,1,34,Russia,Degtyarsk,3,Android,ads
2,3,Asian quake hits European shares\n\nShares in ...,business,200,1,34,Russia,Degtyarsk,3,Android,ads
3,4,India power shares jump on debut\n\nShares in ...,business,200,1,34,Russia,Degtyarsk,3,Android,ads
4,5,Lacroix label bought by US firm\n\nLuxury good...,business,200,1,34,Russia,Degtyarsk,3,Android,ads
...,...,...,...,...,...,...,...,...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie,200,1,34,Russia,Degtyarsk,3,Android,ads
7019,7316,I give this movie 2 stars purely because of it...,movie,200,1,34,Russia,Degtyarsk,3,Android,ads
7020,7317,I cant believe this film was allowed to be mad...,movie,200,1,34,Russia,Degtyarsk,3,Android,ads
7021,7318,The version I saw of this film was the Blockbu...,movie,200,1,34,Russia,Degtyarsk,3,Android,ads


In [2]:
from catboost import CatBoostClassifier

In [29]:
catboost = CatBoostClassifier(max_depth=3, learning_rate=0.1, iterations=100,
                              cat_features=["topic", "gender", "country", "city", "exp_group", "os", "source"])

catboost.fit(X_train.drop(["user_id", "post_id"], axis=1), y_train);

0:	learn: 0.6857634	total: 768ms	remaining: 1m 16s
1:	learn: 0.6845831	total: 1.58s	remaining: 1m 17s
2:	learn: 0.6836716	total: 2.19s	remaining: 1m 10s
3:	learn: 0.6828894	total: 2.79s	remaining: 1m 6s
4:	learn: 0.6822541	total: 3.3s	remaining: 1m 2s
5:	learn: 0.6813594	total: 3.78s	remaining: 59.3s
6:	learn: 0.6808519	total: 4.26s	remaining: 56.6s
7:	learn: 0.6742528	total: 4.73s	remaining: 54.4s
8:	learn: 0.6733138	total: 5.23s	remaining: 52.9s
9:	learn: 0.6681180	total: 5.74s	remaining: 51.7s
10:	learn: 0.6674696	total: 6.21s	remaining: 50.3s
11:	learn: 0.6668603	total: 6.67s	remaining: 48.9s
12:	learn: 0.6628007	total: 7.15s	remaining: 47.9s
13:	learn: 0.6600213	total: 7.64s	remaining: 47s
14:	learn: 0.6573271	total: 8.18s	remaining: 46.3s
15:	learn: 0.6545398	total: 8.66s	remaining: 45.5s
16:	learn: 0.6540365	total: 9.14s	remaining: 44.6s
17:	learn: 0.6531679	total: 9.95s	remaining: 45.3s
18:	learn: 0.6527799	total: 10.6s	remaining: 45s
19:	learn: 0.6525660	total: 11.1s	remaining

In [33]:
X_test.drop(["target"], axis=1, inplace=True)

In [7]:
new_model = CatBoostClassifier().load_model("updated_catboost_model")

In [8]:
X_test["target"] = y_test
X_test["prediction"] = new_model.predict(X_test.drop(["user_id", "post_id"], axis=1))

In [9]:
import numpy as np

users_hitrate = []

for user in X_test["user_id"].unique():
    part = X_test[X_test["user_id"] == user].sort_values("prediction").reset_index()[:5]
    user_hit = 0
    for index, row in part.iterrows():
        if row["prediction"] == 1 and row["target"] == 1:
            user_hit = 1
            break
    users_hitrate.append(user_hit)

print(np.mean(users_hitrate))

0.628483170961962


In [44]:
catboost.save_model("new_catboost_model", format="cbm")

# lightgbm

In [65]:
from lightgbm import LGBMClassifier

light_model = LGBMClassifier(max_depth=3, n_estimators=500)

light_model.fit(X_train, y_train);

In [67]:
light_model.booster_.save_model("new_light_model.txt")

In [68]:
import lightgbm
model_from_load = lightgbm.Booster(model_file="new_light_model.txt")
model_from_load.predict(X_test)

array([0.14912409, 0.09955927, 0.10288893, ..., 0.09607725, 0.09167931,
       0.09607725])

# KNN

In [51]:
import numpy as np

def kernel(distances, h=1):
    
        const = 1 / (np.sqrt(2 * np.pi))
        power = (-1/2) * ((distances)**2) / h**2
    
        return const * np.exp(power)

In [52]:
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler


pipe = Pipeline([('scaler', StandardScaler()), 
                 ('KNN', KNeighborsClassifier(weights=kernel))])

pipe.fit(X_train.drop(["user_id", "post_id"], axis=1), y_train);

# Расчет hitrate@5

In [72]:
import numpy as np

# Создаем DataFrame с прогнозами модели
predictions = pipe.predict(X_test.drop(["user_id", "post_id", "action"], axis=1))

# Добавляем столбец с прогнозами к DataFrame с тестовыми данными
X_test['predicted_action'] = predictions

# Группируем данные по пользователю и выбираем первые пять записей каждого пользователя
grouped = X_test.groupby('user_id').head(5)

# Вычисляем метрику hitrate@5
hits = grouped[(grouped['action'] == 1) & (grouped['action'] == grouped['predicted_action'])]
hitrate = len(hits) / len(grouped['user_id'].unique())

print("Hitrate@5:", hitrate)
